# ABIDES-MARKETS TEST

This is a test for basic use of abides_markets simulation. It uses the original code from [abides-jpmc-public](https://github.com/jpmorganchase/abides-jpmc-public).

In [ ]:
import numpy as np
import pandas as pd
from abides_core import abides
from abides_core.utils import fmt_ts, ns_date, parse_logs_df, str_to_ns
from matplotlib import pyplot as plt

from rohan.abides_wrapper import AbidesWrapper
from rohan.config.simulation_settings import SimulationSettings

## Build runnable configuration

Here we are generating a default config, based on ABIDES RMSC-4. 
The config object is a dictionary containing key elements like, start time, end time, agents to be used, latency and computation delay models.

In [ ]:
config = SimulationSettings()
config.seed = 42
config.date = "20260130"
config.model_dump_json()

In [ ]:
abides_wrapper = AbidesWrapper(config)
abides_config = abides_wrapper.build_configuration()

## Running simulation

Once the config is ready it can be run using the abides runner function.
It instanciates a simulation kernel, runs the configuration and returns an end_state that mostly contains pointers to the different agent objects.
The agents are in their final state, their internal variables can be accessed to extract informations of interests like logs.

In [ ]:
end_state = abides.run(abides_config)

## Retrieving results from end_state

By convention in abides_markets configuration the first agent is the exchange agent. 
It contains an order_book. By default it logs its history.

In [ ]:
order_book = end_state["agents"][0].order_books["ABM"]

### Order book history L1

L1 data snapshots for every tick can be extracted
( best bid and ask price and quantity )

In [ ]:
L1 = order_book.get_L1_snapshots()

Here we plot the time series of the best bid and best ask price thoughout the simulation

In [ ]:
best_bids = pd.DataFrame(L1["best_bids"], columns=["time", "price", "qty"])
best_asks = pd.DataFrame(L1["best_asks"], columns=["time", "price", "qty"])

## All times are in ns from 1970, remove the date component to put them in ns from midnight
best_bids["time"] = best_bids["time"].apply(lambda x: x - ns_date(x))
best_asks["time"] = best_asks["time"].apply(lambda x: x - ns_date(x))

plt.plot(best_bids.time, best_bids.price)
plt.plot(best_asks.time, best_asks.price)

band = 100
plt.ylim(100_000 - band, 100_000 + band)

time_mesh = np.arange(str_to_ns("09:30:00"), str_to_ns("10:10:00"), 1e9 * 60 * 10)
_ = plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

### Order book history L2

L2 data snapshots for every tick can be extracted
( bids and asks price and quantity for every orderbook level. Here max depth logged is a parameter of the simulation and max number of levels we want to retrieve from the orderbook after the simulation is a parameter too)

In [ ]:
L2 = order_book.get_L2_snapshots(nlevels=10)

As an illustration we plot the time series of the fifth best bid price and fifth best ask price throughout the simulation

In [ ]:
## plotting fifth best bid and fifth best ask
times = [t - ns_date(t) for t in L2["times"]]
plt.scatter(times, L2["bids"][:, 5, 0], s=0.5)
plt.scatter(times, L2["asks"][:, 5, 0], s=0.5)

band = 100
plt.ylim(100_000 - band, 100_000 + band)

_ = plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

### Looking at agents logs

All agents can be inspected to retrieve desired information. 
The utility parse_logs_df for instance provides a quick way to retrieve and aggregate the log variables of each agent in a single dataframe

In [ ]:
logs_df = parse_logs_df(end_state)
logs_df.sample(10)

#### Histogram of order submission times for noise agents

As an illustration we retrieve the submission times of all the orders sent by noise agent and display the histogram of all these times

In [ ]:
plt.figure(figsize=(12, 6))

# Extract ORDER_SUBMITTED events for NoiseAgent and convert time_placed to ns from midnight
noise_agent_orders = logs_df[(logs_df.agent_type == "NoiseAgent") & (logs_df.EventType == "ORDER_SUBMITTED")]
noise_agent_times = noise_agent_orders.time_placed.apply(lambda x: pd.Timestamp(x).value - ns_date(pd.Timestamp(x).value))

plt.hist(noise_agent_times, bins=50, alpha=0.7, edgecolor="black")

plt.xlabel("Time (ns from midnight)", fontsize=12)
plt.ylabel("Number of Orders", fontsize=12)
plt.title("NoiseAgent Order Submission Times", fontsize=14, fontweight="bold")

# Set x-axis to match the simulation time window
plt.xlim(str_to_ns("09:30:00"), str_to_ns("10:10:00"))
plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Show summary statistics
print(f"Total NoiseAgent orders: {len(noise_agent_times)}")
if len(noise_agent_times) > 0:
    print(f"First order: {noise_agent_orders.time_placed.min()}")
    print(f"Last order: {noise_agent_orders.time_placed.max()}")

We proceed the same way for value agents as well

In [ ]:
plt.figure(figsize=(12, 6))

# Extract ORDER_SUBMITTED events for ValueAgent and convert time_placed to ns from midnight
value_agent_orders = logs_df[(logs_df.agent_type == "ValueAgent") & (logs_df.EventType == "ORDER_SUBMITTED")]
value_agent_times = value_agent_orders.time_placed.apply(lambda x: pd.Timestamp(x).value - ns_date(pd.Timestamp(x).value))

plt.hist(value_agent_times, bins=50, alpha=0.7, color="orange", edgecolor="black")

plt.xlabel("Time (ns from midnight)", fontsize=12)
plt.ylabel("Number of Orders", fontsize=12)
plt.title("ValueAgent Order Submission Times", fontsize=14, fontweight="bold")

# Set x-axis to match the simulation time window
plt.xlim(str_to_ns("09:30:00"), str_to_ns("10:10:00"))
plt.xticks(time_mesh, [fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Show summary statistics
print(f"Total ValueAgent orders: {len(value_agent_times)}")
if len(value_agent_times) > 0:
    print(f"First order: {value_agent_orders.time_placed.min()}")
    print(f"Last order: {value_agent_orders.time_placed.max()}")